In [9]:
import tensorflow as tf
from Audi_Classifier import Audio_Classifier

In [2]:
data_path = "dataset"
train_data = tf.keras.utils.audio_dataset_from_directory(
    data_path,
    batch_size=8,
    shuffle=True,
    validation_split=0.2,
    subset="training",
    output_sequence_length=48000,
    ragged=False,
    label_mode="categorical",
    labels="inferred",
    sampling_rate=None,
    seed=60

)
validation_data = tf.keras.utils.audio_dataset_from_directory(
    data_path,
    batch_size=8,
    shuffle=True,
    validation_split=0.2,
    subset="validation",
    output_sequence_length=48000,
    ragged=False,
    label_mode="categorical",
    labels="inferred",
    sampling_rate=None,
    seed=5
)


Found 1619 files belonging to 17 classes.
Using 1296 files for training.
Found 1619 files belonging to 17 classes.
Using 323 files for validation.


In [12]:
model = Audio_Classifier.create_model(train_data ,validation_data)
print(model)

model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
        loss='categorical_crossentropy',
        metrics=['accuracy'])

model.fit(train_data, validation_data=validation_data, epochs=30)

None


AttributeError: 'NoneType' object has no attribute 'compile'

In [9]:
model.save('audio_friends.h5')